**Capstone Guideline : Named Entity Recognition Service**

___

Checkpoints : 
1. Instalasi : 1 poin
2. Membuat dan menggunakan model entity recognition : 2 poin
4. Membuat api untuk aplikasi entity recognition dengan tampilan web : 2 poin
5. Membuat api untuk aplikasi entity recognition tanpa tampilan web : 9 poin
6. Melakukan deployment aplikasi pada remote host : 2 poin

# Instalasi 

In [1]:
Untuk membuat flask named entity recognition, kita dapat menggunakan libary : 
- flask : untuk membuat aplikasi / microservice (install : `pip install flask`)
- pandas : tools data analysis, digunakan selama pembuatan fitur aplikasi  (install : `pip install pandas`)
- spacy : model bahasa (install : `pip install spacy`)
- en_core_web_md : kamus bahasa yang akan digunakan spacy (install : `python -m spacy download en_core_web_md` atau `pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-3.0.0/en_core_web_md-3.0.0.tar.gz`)

- requests : tools untuk mengirim dan menerima paket, digunakan untuk menguji aplikasi yang sudah dibuat (built-in python)

SyntaxError: invalid syntax (<ipython-input-1-9153f9b24ae7>, line 1)

In [2]:
from flask import Flask,render_template,url_for,request
import pandas as pd
import spacy
from spacy import displacy
import en_core_web_md
import requests

# Membuat dan Menggunakan Model NER

Sebelum memodelkan bahasa /teks, hal pertama yang dibutuhkan adalah membuat model tersebut. Spacy memudahkan kita dengan memberikan model yang sudah dibuat. 

Jalankan sintaks berikut untuk meload model bahasa tersebut

In [6]:
# load model bahasa spacy
nlp = en_core_web_md.load()

Setelah memiliki model, komponen penting berikutnya dalah data yang akan dimodelkan. Dalam hal ini, kami memberi contoh sample buku Sherlock Holmes dan teks random sebagai data yang akan dimodelkan. 

In [11]:
# load teks sherlock holmes
# f = open('sherlock_holmes.txt', 'r')
# sherlock_holmes = f.read().replace('\n', ' ')
# f.close()

# jika tidak ingin teks yang panjang, bisa gunakan contoh teks sendiri 
text = """This text is an example used for Algoritma's Capstone API Project. 
You can use any named entitiy such person name like Nicola Tesla, Steve Jobs, or even Elon Musk. 
Please mind that the model is trained on formal writing, thus slang will less likely to be recognized correctly."""


Untuk menggunakan model Entity Recognition milik spacy, cukup masukkan teks sebagai parameter dalam membuat object bahasa seperti contoh berikut: 

In [12]:
doc = nlp(text)

Untuk menampilkan nilai dari teks tersebut, panggil objek `doc` kembali

In [13]:
doc

This text is an example used for Algoritma's Capstone API Project. 
You can use any named entitiy such person name like Nicola Tesla, Steve Jobs, or even Elon Musk. 
Please mind that the model is trained on formal writing, thus slang will less likely to be recognized correctly.

In [14]:
doc.ents
#untuk melihat semua entitas yang dimodelkan

(Algoritma, Capstone API Project, Nicola Tesla, Steve Jobs)

In [15]:
doc.ents[0].label_
#untuk mengakses kategori dari entitas tersebut

'ORG'

Untuk mengakes seluruh element entitas , jalankan sintaks berikut : 


In [16]:
# membuat pasangan label dan nilai entitas
d = [(ent.label_, ent.text) for ent in doc.ents]

# transform pasangan label menjadi dataframe 
df = pd.DataFrame(d, columns=['category', 'value'])

In [17]:
df

,category,value
0,ORG,Algoritma
1,ORG,Capstone API Project
2,PERSON,Nicola Tesla
3,PERSON,Steve Jobs


# Membuat API NER dengan tampilan Web 

## app.py

Untuk membuat aplikasi NER dengan flask, silakan **lengkapi code berikut** dan salin kedalam file `app.py`.

In [9]:
from flask import Flask,render_template,url_for,request, jsonify
import re
import pandas as pd
import spacy
from spacy import displacy
import en_core_web_md
import json 

# @TASK : Load model bahasa 
nlp = spacy.load('en_core_web_md')
# END OF TASK 

app = Flask(__name__)

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/process',methods=["POST"])
def process():
    if request.method == 'POST':
        choice = request.form['taskoption']
        rawtext = request.form['rawtext']
        doc = nlp(rawtext)
        d = []
        if len(doc.ents) > 0:
            for ent in doc.ents:
                d.append((ent.label_, ent.text))
                df = pd.DataFrame(d, columns=['category', 'value'])

                # @TASK : COMPLETE THE FOLLOWING CODES
                ORG_named_entity = df.query('category' == 'ORG')['value'] # Subset semua entitas dengan kategori 'ORG'
                PERSON_named_entity = df.query('category' =='PERSON')['value'] # Subset semua entitas dengan kategori 'PERSON'
                GPE_named_entity = df.query('category' =='GPE')['value'] # Subset semua entitas dengan kategori 'GPE'
                MONEY_named_entity = df.query('category'=='MONEY')['value'] # Subset semua entitas dengan kategori 'MONEY'
                # END OF TASK 

            if choice == 'organization':
                results = ORG_named_entity
                num_of_results = len(results)
            elif choice == 'person':
                results = PERSON_named_entity
                num_of_results = len(results)
            elif choice == 'geopolitical':
                results = GPE_named_entity
                num_of_results = len(results)
            elif choice == 'money':
                results = MONEY_named_entity
                num_of_results = len(results)
            elif choice == 'Select Category':
                results = pd.DataFrame()
                num_of_results = len(results)
        else:
            results = pd.DataFrame()
            num_of_results = len(results)

    return render_template("index.html",results=results,num_of_results = num_of_results, original_text = rawtext)


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Restarting with windowsapi reloader


SystemExit: 1

## menjalankan app.py

Untuk menjalankan app.py, silakan buka terminal/console/anaconda prompt menuju direktori tempat app.py berada, lalu jalnkan sintaks 
`python app.py` .

Jika code berjalan dengan baik, silakan buka laman yang dirujuk pada console (biasanya http://127.0.0.1:5000/) melalui browser dan aplikasi akan berjalan sesuai tampilan berikut : 

![](assets/default_homepage.png)

Setelah aplikasi berjalan, silakan inputkan teks dan pilih kategori yang akan diekstrak, lalu pilih submit. Jika tidak ada eror, maka form result akan memberikan hasil kembalian berupa banyaknya entitas yang dikenali dan list entitas tersebut. 

# Membuat API NER tanpa tampilan WEB

## Membuat Endpoint

Dalam web service, endpoint adalah sebuah alamat spesifik yang ditujukan untuk kebutuhan khusus. 
Sebagai contoh, berikut adalah beberapa endpoint web API milik spotify : 
- https://api.spotify.com/v1/albums{id} : Digunakan untuk mengakses album dengan album id = id
- https://api.spotify.com/v1/artists/{id} : digunakan untuk mengakses artist dengan artist id = id
- https://api.spotify.com/v1/tracks/{id} : digunakan untuk mengakses tracks dengan track id = id

Dalam flask, endpoint dinotasikan dengan `@app.route('alamat endpoint')`, diikuti dengan fungsi yang akan dijalankan. 
Sebelumnya, dalam `app.py`, kita sudah membuat sebuah endpoint home ('/') dengan sintaks : 
```python
@app.route('/')
def index():
    return render_template("index.html")
```    

Menggunakan sintaks diatas, ketika user mengakses aplikasi kita di menu utama, maka fungsi `index()` akan dijalankan. 

Sebagai contoh tambahan, coba salin sintaks berikut kedalam `app.py` dan coba akses endpoint tersebut 

```python
@app.route('/endpoint_tertentu')
def nama_fungsi_tertentu():
    # secara teknis, kita dapat melakukan apapun dalam fungsi ini 
    return ("Fungsi ini akan dijalankan saat endpoint tersebut diakses")
```


**notes**: nama fungsi dalam aplikasi flask harus unik. 

## Endpoint Methods

Senada dengan konsep CRUD (Create, Read, Update, Delete) dalam database, dalam API konsep tersebut diadopsi menjadi :
- POST(CREATE),
- GET(READ),
- PUT(UPDATE),
- DELETE

Secara konsep, method POST, GET, PUT, dan DELETE dibuat untuk mengatur API agar dapat lebih terstruktur. Meskipun pada prakterknya, endpoint yang kita definisikan menggunakan method "POST" tidaklah harus untuk membuat data. Artinya, tidak ada aturan yang mengharuskan kita untuk menuliskan jenis method dalam setiap endpoints kita (default endpoints menggunakan method GET).

Dalam capstone ini, kita hanya akan berfokus pada 2 jenis method, GET dan POST. Sebagai contoh tambahan, silakan salin sintaks enspoint berikut kedalma `app.py`, lalu jalankan ulang aplikasinya dan lihat perbedaan saat melakukan akses terhadap kedua endpoint tersebut. 

```python
@app.route('/endpoint_get', methods=['GET'])
def contoh_get():
    return ("Contoh endpoint get")
    
@app.route('/endpoint_post', methods=['POST'])
def contoh_post():
    return ("Contoh endpoint post")
    
@app.route('/endpoint_multi', methods=['GET', 'POST'])
def multi_method():
    if request.method == 'POST':
        return ("Nilai ini akan dikembalikan jika endpoint ini diakses dengan method POST")
    else : 
        return ("Nilai ini akan dikembalikan jika endpoint ini diakses dengan method GET")
```


**Note : Secara default, mengakses endpoint melaui browser akan menggunakan method 'GET'**. 

Untuk mempermudah pengujian, disarankan untuk menggunakan tools lain seperti "postman", atau menggunakan python. 


## Mengakses endpoint menggunakan python

Dalam banyak kasus, web API tidak dipanggil melalui browser, melainkan pada level aplikasi / code. Untuk mengakses web api menggunakan python, gunakan `curl` atau `requests`. Code berikut akan mengakses endpoint yang sama dengan method yang berbeda: 

In [3]:
import requests
response_post = requests.request(method='POST', url='http://localhost:5000/endpoint_multi')
response_get = requests.request(method='GET', url='http://localhost:5000/endpoint_multi')

Respon dari sebuah service web API dibungkus dalam http response. Jika objek `response_post` dipanggil, maka yang tampil adalah nilai kode status repons tersebut (silakan merujuk referensi HTTP response code untuk informasi lebih lengkap) : 

In [5]:
response_post

<Response [200]>

In [6]:
# secara eksplisit mengambil atribut status code dari objek response
response_post.status_code

200

Untuk dapat mengambil konten dari nilai yang diterima, akses atribut .text atau .json jika nilai yang diterima berupa json

In [7]:
response_post.text

'Nilai ini akan dikembalikan jika endpoint ini diakses dengan method POST'

In [8]:
response_get.text

'Nilai ini akan dikembalikan jika endpoint ini diakses dengan method GET'

Referensi : [HTTP response code, by mozilla](https://developer.mozilla.org/id/docs/Web/HTTP/Status)

## Mengirim data ke endpoint

Untuk melakukan ekstraksi entitas terhadap data yang kita miliki pada layanan web api, kita harus mengirimkan data tersebut melalui protokol http. 
Method API yang biasa digunakan dalam melayani request tersebut adalah POST. 
Agar data dapat dikirim dengan baik, data harus dikirim dalam bentuk objek JSON (JSON dalam python = Dictionary). 


Sebagai contoh, sintaks berikut akan mengirimkan data json kedalam endpoint yang dituju. 

In [10]:
# membuat data yang akan dikirim (dalam bentuk dictionary)
data_yang_akan_dikirim = {
    'nama' : 'Steve', 
    'usia' : 12, 
    'pekerjaan' : 'Data Scientist'
}

# endpoint yang akan dituju
endpoint = 'http://localhost:5000/tes_send_json'

# mengeksekusi request
response = requests.request(method='POST', url=endpoint, json=data_yang_akan_dikirim)

In [11]:
response

<Response [404]>

Jika dijalankan, sintaks diatas tentu akan menghasilkan error. Hal ini dikarenakan pada aplikasi kita belum terdapat endpoint `'/send_json'`. 
Ikuti langkah berikutnya untuk membuat endpoint yang dapat menerima objek yang kita kirim tersebut 

## Membuat endpoint yang menerima data

Untuk membaca data yang dikirimkan pada endpoint tertentu, kita dapat menggunakan method method `request.get_json()` dari flask. (Silakan merujuk ke app.py dan pastikan terdapat sintaks `from flask import request`. 

Silakan tambahkan endpoint dibawah ini pada `app.py` dan jalankan kembali request pada segmen sebelumnya

```python
@app.route('/tes_send_json', methods=['POST'])
def tes_send_json():
    data = request.get_json() # proses membaca json yang dikirim 
    nama = data['nama']
    usia = data['usia']
    pekerjaan = data['pekerjaan']
    
    return ("Halo, {nama}. Usiamu adalah {usia} dan pekerjaanmu adalah {pekerjaan}".format(nama=nama, usia=usia, pekerjaan=pekerjaan))
```

In [16]:
# coba jalankan request di segment sebelumnya

data_yang_akan_dikirim = {
    'nama' : 'Steve', 
    'usia' : 12, 
    'pekerjaan' : 'Data Scientist'
}

endpoint = 'http://localhost:5000/tes_send_json'

response = requests.request(method='POST', url=endpoint, json=data_yang_akan_dikirim)


In [17]:
response.text

'Halo, Steve. Usiamu adalah 12 dan pekerjaanmu adalah Data Scientist'

## Mengembalikan hasil pemrosesan sebagai objek JSON dari endpoint

Dalam beberapa kasus, analisis atau pengelolaan yang kita lakukan berakhir pada tipe data DataFrame milik pandas. Untuk mengubah DataFrame menjadi json, gunakan method `.to_json()`. Berikut adalah contoh endpoint yang mengembalikan dataframe dalam bentuk json.

```python
@app.route('/tes_return_json', methods=['POST'])
def tes_return_json():
    data = request.get_json() # proses membaca json yang dikirim 
    df = pd.DataFrame([data]) # mengolah data menjadi dataframe

    return (df.to_json()) # mengembalikan dataframe dalam bentuk json
```

In [18]:
data_yang_akan_dikirim = {
    'nama' : 'Steve', 
    'usia' : 12, 
    'pekerjaan' : 'Data Scientist'
}

endpoint = 'http://localhost:5000/tes_return_json'

response = requests.request(method='POST', url=endpoint, json=data_yang_akan_dikirim)

In [19]:
response.text

'{"nama":{"0":"Steve"},"usia":{"0":12},"pekerjaan":{"0":"Data Scientist"}}'

In [20]:
response.json()

{'nama': {'0': 'Steve'},
 'usia': {'0': 12},
 'pekerjaan': {'0': 'Data Scientist'}}

In [25]:
import pandas as pd
pd.DataFrame(response.json())

,nama,usia,pekerjaan
0,Steve,12,Data Scientist


## Task : Endpoint 

Sebagai tugas dalam capstone ini, buatlah endpoint yang dapat mengembalikan **semua** jenis entitas dari teks yang dikirim. 

Berikut adalah spesifikasi singkat dari endpoint tersebut
- Input : Teks dalam bentuk JSON, dengan raw teks disimpan dalam elemen "text" seperti contoh berikut: 
```python
{
    "text" : "ini adalah contoh teks yang dikirim"
}
```
- Output : Bentuk JSON dari dataframe yang berisi 2 kolom (kategori entitas, value). 

Untuk menyelesaikan task tersebut silakan lengkapi code endpoint berikut lalu tambahkan pada `app.py`

In [7]:
@app.route('/get_entities', methods=['POST'])
def get_entities():
    
    # ambil data dari json yang diterima endpoint
    data = request.get_json()
   
    # ambil nilai teks dari data
    text = data['text']
    
    # modelkan teks dengan model scipy 
    doc = en_core_web_md.load(text)
    
    # membuat pasangan label dan nilai entitas
    d = [(ent.label_,ent.text) for ent in doc.ents]

    # transform pasangan label menjadi dataframe 
    df = pd.DataFrame(d,columns=['category','value'])
    
    return (df.to_json())

NameError: name 'app' is not defined

In [10]:
# Jalankan sintaks berikut untuk menguji endpoint tersebut

text = """This text is an example used for Algoritma's Capstone API Project. 
You can use any named entitiy such person name like Nicola Tesla, Steve Jobs, or even Elon Musk. 
You can also use organization name such Algoritma, Facebook, Apple."""

data = {"text": text}
url = 'http://localhost:5000/get_entities'
response = requests.request(method='POST', url=url, json=data)
response.json()

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /get_entities (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000292C1EDE250>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

## Task Endpoint (bonus)

Jika dilihat dari hasil kembalian endpoint sebelumnya nilai return JSON yang dihasilkan sedikit tidak lazim, dimana sewajarnya nilai setiap entitas dikelompokkan sebagai values kedalam setiap kategori. 


**Bonus Task** : Buatlah endpoint `'/get_entities_normalized'` yang menghasilkan JSON dengan key jenis kategori entitas, dan value berupa list dari entitas dengan kategori yang bersangkutan. Berikut ilustrasi dari nilai return endpoint yang diinginkan :  

```python
{category1 : [value1, value2, ... , valuen],
 category2 : [value1, value2, ... , valuen],}
```

In [51]:
# Jalankan sintaks ini untuk menguji endpoint tersebut
url = 'http://localhost:5000/get_entities_normalized'
response = requests.request(method='POST', url=url, json=data)
response.json()

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /get_entities_normalized (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000254B64C4A90>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

# Deployment ke remote host

## Upload code ke git repository (github)

1. Membuat repositori baru 
2. Mengupload file ke repositori baru dan commit changes

## Create Heroku App

1. Membuat aplikasi baru
2. Menyambungkan repositori github kedalam aplikasi baru 
3. Melakukan deployment